## Import libraries

In [1]:
import requests
import os
import cv2
import requests
import mediapipe as mp
import time
import csv
import pandas as pd

## Initial Setup: Checking ESP32 Endpoint Accessibility

Ensure that the ESP32 endpoints are accessible. This part of the notebook performs the following tasks:

1. **Define ESP32 Endpoints**:
   - The IP address of the ESP32 is specified.
   - All necessary endpoints are defined in a dictionary for easy management.

2. **Accessibility Check**:
   - Each endpoint is checked using an HTTP `GET` request.
   - The response is logged:
     - ✅ Indicates the endpoint is accessible and responding correctly.
     - ❌ Indicates a connection issue or unexpected HTTP status code.

3. **Timeout Handling**:
   - A timeout of 5 seconds is set for each request to prevent the script from hanging indefinitely.
---


In [2]:
# ESP32 IP address
esp32_ip = "127.0.0.1:5000"
#esp32_ip = "192.168.1.25"

# Define endpoints with their respective HTTP methods
endpoints = {
    "motorAngles": {"url": f"http://{esp32_ip}/motorAngles", "method": "GET"},
    "pythonStarted": {"url": f"http://{esp32_ip}/pythonStarted", "method": "GET"},
    "datasetComplete": {"url": f"http://{esp32_ip}/datasetComplete", "method": "GET"},
    "combinationProcessed": {"url": f"http://{esp32_ip}/combinationProcessed", "method": "GET"},
    "setMotorAngles": {"url": f"http://{esp32_ip}/setMotorAngles", "method": "POST"},
    "setPythonStarted": {"url": f"http://{esp32_ip}/setPythonStarted", "method": "POST"},
    "setCombinationProcessed": {"url": f"http://{esp32_ip}/setCombinationProcessed", "method": "POST"}
}

In [3]:
def get_esp32_variable(endpoint_key):
    # Retrieve the endpoint details
    if endpoint_key not in endpoints:
        print(f"❌ Endpoint key '{endpoint_key}' not found in the endpoints dictionary.")
        return None

    endpoint = endpoints[endpoint_key]
    url = endpoint["url"]
    method = endpoint["method"]

    # Ensure the method is GET
    if method != "GET":
        print(f"❌ Incorrect method for {endpoint_key} endpoint. Expected GET, got {method}.")
        return None

    # Send the GET request
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            print(f"✅ {endpoint_key} fetched successfully: {response.text}")
            return response.text
        else:
            print(f"❌ Error fetching {endpoint_key}: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Connection error while fetching {endpoint_key}: {e}")
        return None


In [4]:
def set_esp32_variable(endpoint_key, variable_name, value):
    # Retrieve the endpoint details
    if endpoint_key not in endpoints:
        print(f"❌ Endpoint key '{endpoint_key}' not found in the endpoints dictionary.")
        return False

    endpoint = endpoints[endpoint_key]
    url = endpoint["url"]
    method = endpoint["method"]

    # Ensure the method is POST
    if method != "POST":
        print(f"❌ Incorrect method for {endpoint_key} endpoint. Expected POST, got {method}.")
        return False

    # Send the POST request
    try:
        response = requests.post(url, data={variable_name: str(value)}, timeout=10)
        if response.status_code == 200:
            print(f"✅ {variable_name} set to {value} successfully.")
            return True
        else:
            print(f"❌ Error setting {variable_name}: {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"❌ Connection error while setting {variable_name}: {e}")
        return False


### Optional: test set and get functions

In [5]:
# Test the get_esp32_variable function
print("Testing get_esp32_variable...\n")

# Test fetching motorAngles
motor_angles_raw = get_esp32_variable("motorAngles")
if motor_angles_raw is not None:
    try:
        motor_angles = [float(angle) for angle in motor_angles_raw.split(", ")]
        print(f"✅ motorAngles fetched successfully: {motor_angles}")
    except ValueError as e:
        print(f"❌ Failed to parse motorAngles: {e}")
else:
    print("❌ Failed to fetch motorAngles.")

# Test fetching datasetComplete
dataset_complete_raw = get_esp32_variable("datasetComplete")
if dataset_complete_raw is not None:
    dataset_complete = dataset_complete_raw == "True"
    print(f"✅ datasetComplete fetched: {dataset_complete}")
else:
    print("❌ Failed to fetch datasetComplete.")

# Test fetching pythonStarted
python_started_raw = get_esp32_variable("pythonStarted")
if python_started_raw is not None:
    python_started = python_started_raw == "True"
    print(f"✅ pythonStarted fetched: {python_started}")
else:
    print("❌ Failed to fetch pythonStarted.")

# Test fetching combinationProcessed
combination_processed_raw = get_esp32_variable("combinationProcessed")
if combination_processed_raw is not None:
    combination_processed = combination_processed_raw == "True"
    print(f"✅ combinationProcessed fetched: {combination_processed}")
else:
    print("❌ Failed to fetch combinationProcessed.")

# Test invalid endpoint key
invalid_endpoint = get_esp32_variable("invalidEndpoint")
if invalid_endpoint is None:
    print("✅ Properly handled invalid endpoint.")


Testing get_esp32_variable...

✅ motorAngles fetched successfully: -10.0, -10.0, -10.0, -10.0, -10.0
✅ motorAngles fetched successfully: [-10.0, -10.0, -10.0, -10.0, -10.0]
✅ datasetComplete fetched successfully: False
✅ datasetComplete fetched: False
✅ pythonStarted fetched successfully: False
✅ pythonStarted fetched: False
✅ combinationProcessed fetched successfully: False
✅ combinationProcessed fetched: False
❌ Endpoint key 'invalidEndpoint' not found in the endpoints dictionary.
✅ Properly handled invalid endpoint.


In [ ]:
# Test setting pythonStarted
print("Testing set_esp32_variable...\n")
success = set_esp32_variable("setPythonStarted", "pythonStarted", True)
print(f"✅ Test result for pythonStarted: {success}")

# Test setting combinationProcessed
success = set_esp32_variable("setCombinationProcessed", "combinationProcessed", True)
print(f"✅ Test result for combinationProcessed: {success}")

# Test invalid endpoint key
success = set_esp32_variable("invalidEndpoint", "dummyVariable", True)
print(f"✅ Test result for invalid endpoint: {success}")


In [6]:
# Test setting pythonStarted
print("Testing set_esp32_variable...\n")
success = set_esp32_variable("setPythonStarted", "pythonStarted", True)
print(f"✅ Test result for pythonStarted: {success}")

Testing set_esp32_variable...

✅ pythonStarted set to True successfully.
✅ Test result for pythonStarted: True


In [14]:
# Test setting combinationProcessed
success = set_esp32_variable("setCombinationProcessed", "combinationProcessed", True)
print(f"✅ Test result for combinationProcessed: {success}")

✅ combinationProcessed set to True successfully.
✅ Test result for combinationProcessed: True


## HandLandmarker

In [24]:
# Define output paths for the original and annotated videos within 'roboticHand_data' folder
# and the CSV file path
output_folder = 'raw_dataset'
csv_file_path = os.path.join(output_folder, output_folder + ".csv")

os.makedirs(output_folder, exist_ok=True)
original_video_path = os.path.join(output_folder, 'cam_video_original.mp4')
annotated_video_path = os.path.join(output_folder, 'cam_video_annotated.mp4')

# Initialize dataset
dataset = []

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hand_detector = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Start capturing from the laptop camera
cap = cv2.VideoCapture(0)
fps = 20  # Adjust FPS as needed
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define codecs and VideoWriter objects for both videos
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_original = cv2.VideoWriter(original_video_path, fourcc, fps, (width, height))
out_annotated = cv2.VideoWriter(annotated_video_path, fourcc, fps, (width, height))


In [25]:
# Generate column headers for CSV
keypoints = [
    "WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP", 
    "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP",
    "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP",
    "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP",
    "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP"
]
keypoint_columns = [f"{i}_{name}" for i, name in enumerate(keypoints)]
world_keypoint_columns = [f"{i}_world_{name}" for i, name in enumerate(keypoints)]

# Define the DataFrame and column headers
columns = ["Sample","Num_Attempts","Num_Hands", "Is_Left", "Frame", "Motor_1", "Motor_2", "Motor_3", "Motor_4", "Motor_5"] + world_keypoint_columns + keypoint_columns
dataframe = pd.DataFrame(columns=columns)

In [26]:
# Variable to track the previous motor angles
previous_motor_angles = [-10.0, -10.0, -10.0, -10.0, -10.0]
successful_detection = False
trial_limit = 10
trial_counter = trial_limit
sample_limit = 4
sample_counter = 0

# If pythonStarted is False, set it to True
python_started_raw = get_esp32_variable("pythonStarted")
if python_started_raw is not None:
    python_started = python_started_raw == "True"
    if not python_started:
        set_esp32_variable("setPythonStarted", "pythonStarted", True)
        print("✅ Python started flag set to True.")

# Define a folder to save the extracted frames
frames_folder = os.path.join(output_folder, "frames")
os.makedirs(frames_folder, exist_ok=True)

frame_counter = 0  # Counter to name frames uniquely

# Processing loop
try:
    while True:
        # Capture frame from the camera
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting...")
            break

        # Save original frame
        out_original.write(frame)

        if trial_counter==0:
            print('5')
            # Construct the row for unsuccessful detection
            row = [
                0,
                trial_limit - trial_counter, # Num_Attempts
                -1,  # Num_Hands
                False,  # Is_Left
                frame_counter,
                *motor_angles,  # Motor angles
                *[[0] for i in range(21)],  # Keypoint coordinates
                *[[0] for i in range(21)]  # World keypoint coordinates
            ]
            print(row)
            dataframe.loc[len(dataframe)] = row  # Append row to the DataFrame
            trial_counter = trial_limit
            sample_counter = 0
            # Set combinationProcessed to True
            set_esp32_variable("setCombinationProcessed", "combinationProcessed", True)
        else:
            # Fetch motor angles
            motor_angles_raw = get_esp32_variable("motorAngles")
            if motor_angles_raw is not None:
                try:
                    motor_angles = [float(angle) for angle in motor_angles_raw.split(", ")]
                except ValueError as e:
                    print(f"❌ Failed to parse motorAngles: {e}")
                    continue

                # Check if the current combination is different from the previous one
                new_angles = (motor_angles != previous_motor_angles)  
                print('1')            
                if (not new_angles and (not successful_detection)) or new_angles:
                    # Convert the frame to RGB for MediaPipe processing
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    # Process the frame to detect hands
                    result = hand_detector.process(frame_rgb)
                    print('2')
                    # Draw hand landmarks if hands are detected
                    if result.multi_hand_landmarks:
                        print('3')
                        print(f"Hand detected for motor angles: {motor_angles}")
                        sample_counter+=1
                        for hand_landmarks in result.multi_hand_landmarks:
                            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                            # Save to CSV
                            with open(csv_file_path, mode="a", newline="") as csv_file:
                                writer = csv.writer(csv_file)
                                for idx in range(len(result.multi_handedness)):
                                    # Extract hand data
                                    is_left = result.multi_handedness[idx].classification[0].label == "Left"
                                    keypoints = [[i.x,i.y,i.z] for i in result.multi_hand_landmarks[idx].landmark]
                                    world_keypoints = [[i.x,i.y,i.z] for i in result.multi_hand_world_landmarks[idx].landmark]
                                    
                                    # Construct the row
                                    row = [
                                        sample_counter,
                                        trial_limit - trial_counter, # Num_Attempts
                                        idx + 1,  # Num_Hands
                                        is_left,  # Is_Left
                                        frame_counter, # Frame counter
                                        *motor_angles,  # Motor angles
                                        *world_keypoints,  # World keypoint coordinates
                                        *keypoints,  # Keypoint coordinates
                                    ]
                                    print(row)
                                    dataframe.loc[len(dataframe)] = row  # Append row to the DataFrame
                        if sample_counter == sample_limit:
                            trial_counter = trial_limit
                            sample_counter = 0
                            # Set combinationProcessed to True
                            set_esp32_variable("setCombinationProcessed", "combinationProcessed", True)
                            #successful_detection = True
                    else:
                        print('4')
                        trial_counter-=1
                        successful_detection = False
                        print(f"Cannot detect a hand yet for {motor_angles}")
                    previous_motor_angles = motor_angles
                

        # Save each frame as an image
        frame_filename = os.path.join(frames_folder, f"frame_{frame_counter:04d}.png")
        cv2.imwrite(frame_filename, frame)        

        # Save the annotated frame in video
        out_annotated.write(frame)

        # Display the annotated frame only
        cv2.imshow("Annotated Camera Feed", frame)

        frame_counter += 1

        # Fetch datasetComplete
        dataset_complete_raw = get_esp32_variable("datasetComplete")
        if dataset_complete_raw is not None and dataset_complete_raw == "True":
            print("Dataset complete. Stopping detection process...")
            break

        # Check if 'q' is pressed for manual stop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Manual stop detected. Ending process...")
            break


    # Cleanup
    cap.release()
    out_original.release()
    out_annotated.release()
    try:
        hand_detector.close()
    except Exception as e:
        print(f"Error during hand_detector closing: {e}")

    cv2.destroyAllWindows()

    # Set pythonStarted to False
    set_esp32_variable("setCombinationProcessed", "combinationProcessed", True)
    set_esp32_variable("setPythonStarted", "pythonStarted", False)

    # Save the DataFrame to a CSV file
    # csv_file_path = os.path.join(output_folder, "hand_detection_results.csv")
    dataframe.to_csv(csv_file_path, index=False)
    print(f"Dataset saved to {csv_file_path}")
except Exception as e:
    print(f"Error during processing: {e}")

✅ pythonStarted fetched successfully: False
✅ pythonStarted set to True successfully.
✅ Python started flag set to True.
✅ motorAngles fetched successfully: -10.0, -10.0, -10.0, -10.0, -10.0
1
2
4
Cannot detect a hand yet for [-10.0, -10.0, -10.0, -10.0, -10.0]
✅ datasetComplete fetched successfully: False
✅ motorAngles fetched successfully: -10.0, -10.0, -10.0, -10.0, -10.0
1
2
4
Cannot detect a hand yet for [-10.0, -10.0, -10.0, -10.0, -10.0]
✅ datasetComplete fetched successfully: False
✅ motorAngles fetched successfully: -10.0, -10.0, -10.0, -10.0, -10.0
1
2
4
Cannot detect a hand yet for [-10.0, -10.0, -10.0, -10.0, -10.0]
✅ datasetComplete fetched successfully: False
✅ motorAngles fetched successfully: -10.0, -10.0, -10.0, -10.0, -10.0
1
2
4
Cannot detect a hand yet for [-10.0, -10.0, -10.0, -10.0, -10.0]
✅ datasetComplete fetched successfully: False
✅ motorAngles fetched successfully: -10.0, -10.0, -10.0, -10.0, -10.0
1
2
4
Cannot detect a hand yet for [-10.0, -10.0, -10.0, -10.